# 1. Import and Install Dependencies

In [1]:
!pip install tensorflow==2.12.0 opencv-python mediapipe scikit-learn matplotlib pygame

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import sys
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


# 2. Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 4. Setup Folders for Collection

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['No-Pose', 'L-Pose', 'X-Pose', 'Namaste-Pose'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [9]:
for action in actions: 
    #dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    #for sequence in range(1,no_sequences+1):
     for sequence in range(no_sequences):
        try: 
            #os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [10]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    # cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [11]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
label_map

{'No-Pose': 0, 'L-Pose': 1, 'X-Pose': 2, 'Namaste-Pose': 3}

In [12]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [13]:
# Ok to skip
np.array(sequences).shape

(120, 30, 1662)

In [14]:
# Ok to skip
np.array(labels).shape

(120,)

In [15]:
# Ok to skip
X = np.array(sequences)

In [16]:
# Ok to skip
X.shape

(120, 30, 1662)

In [17]:
# Ok to skip
y = to_categorical(labels).astype(int)

In [18]:
# Ok to skip
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [19]:
# Ok to skip
y_test.shape

(6, 4)

# 7. Build and Train LSTM Neural Network

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [24]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [25]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [26]:
model.fit(X_train, y_train, epochs=1000)

Epoch 1/1000
4/4 [==============================] - 6s 68ms/step - loss: 1.4212 - categorical_accuracy: 0.2368
Epoch 2/1000
4/4 [==============================] - 0s 67ms/step - loss: 1.4019 - categorical_accuracy: 0.2368
Epoch 3/1000
4/4 [==============================] - 0s 68ms/step - loss: 1.6170 - categorical_accuracy: 0.2193
Epoch 4/1000
4/4 [==============================] - 0s 69ms/step - loss: 1.3890 - categorical_accuracy: 0.2281
Epoch 5/1000
4/4 [==============================] - 0s 70ms/step - loss: 1.3946 - categorical_accuracy: 0.3158
Epoch 6/1000
4/4 [==============================] - 0s 68ms/step - loss: 1.6350 - categorical_accuracy: 0.2018
Epoch 7/1000
4/4 [==============================] - 0s 69ms/step - loss: 1.3825 - categorical_accuracy: 0.2456
Epoch 8/1000
4/4 [==============================] - 0s 72ms/step - loss: 1.3711 - categorical_accuracy: 0.2368
Epoch 9/1000
4/4 [==============================] - 0s 68ms/step - loss: 1.3226 - categorical_accuracy: 0.4737
E

4/4 [==============================] - 0s 69ms/step - loss: 0.3333 - categorical_accuracy: 0.8509
Epoch 147/1000
4/4 [==============================] - 0s 67ms/step - loss: 0.4557 - categorical_accuracy: 0.7982
Epoch 148/1000
4/4 [==============================] - 0s 61ms/step - loss: 1.2506 - categorical_accuracy: 0.5965
Epoch 149/1000
4/4 [==============================] - 0s 65ms/step - loss: 0.9241 - categorical_accuracy: 0.5526
Epoch 150/1000
4/4 [==============================] - 0s 63ms/step - loss: 0.6296 - categorical_accuracy: 0.7456
Epoch 151/1000
4/4 [==============================] - 0s 67ms/step - loss: 0.6054 - categorical_accuracy: 0.8596
Epoch 152/1000
4/4 [==============================] - 0s 67ms/step - loss: 0.6185 - categorical_accuracy: 0.7018
Epoch 153/1000
4/4 [==============================] - 0s 65ms/step - loss: 0.4934 - categorical_accuracy: 0.7895
Epoch 154/1000
4/4 [==============================] - 0s 64ms/step - loss: 0.4880 - categorical_accuracy: 0.780

4/4 [==============================] - 0s 68ms/step - loss: 0.0890 - categorical_accuracy: 0.9649
Epoch 291/1000
4/4 [==============================] - 0s 67ms/step - loss: 0.1046 - categorical_accuracy: 0.9474
Epoch 292/1000
4/4 [==============================] - 0s 71ms/step - loss: 0.1526 - categorical_accuracy: 0.9386
Epoch 293/1000
4/4 [==============================] - 0s 65ms/step - loss: 0.0720 - categorical_accuracy: 0.9737
Epoch 294/1000
4/4 [==============================] - 0s 72ms/step - loss: 0.0984 - categorical_accuracy: 0.9474
Epoch 295/1000
4/4 [==============================] - 0s 75ms/step - loss: 0.0757 - categorical_accuracy: 0.9737
Epoch 296/1000
4/4 [==============================] - 0s 67ms/step - loss: 0.1112 - categorical_accuracy: 0.9474
Epoch 297/1000
4/4 [==============================] - 0s 65ms/step - loss: 0.1230 - categorical_accuracy: 0.9298
Epoch 298/1000
4/4 [==============================] - 0s 76ms/step - loss: 0.1137 - categorical_accuracy: 0.947

4/4 [==============================] - 0s 63ms/step - loss: 0.0283 - categorical_accuracy: 0.9912
Epoch 435/1000
4/4 [==============================] - 0s 69ms/step - loss: 0.0587 - categorical_accuracy: 0.9737
Epoch 436/1000
4/4 [==============================] - 0s 68ms/step - loss: 0.0400 - categorical_accuracy: 0.9825
Epoch 437/1000
4/4 [==============================] - 0s 67ms/step - loss: 0.0390 - categorical_accuracy: 0.9825
Epoch 438/1000
4/4 [==============================] - 0s 66ms/step - loss: 0.0540 - categorical_accuracy: 0.9825
Epoch 439/1000
4/4 [==============================] - 0s 64ms/step - loss: 0.0156 - categorical_accuracy: 1.0000
Epoch 440/1000
4/4 [==============================] - 0s 67ms/step - loss: 0.0283 - categorical_accuracy: 1.0000
Epoch 441/1000
4/4 [==============================] - 0s 66ms/step - loss: 0.0738 - categorical_accuracy: 0.9737
Epoch 442/1000
4/4 [==============================] - 0s 67ms/step - loss: 0.4928 - categorical_accuracy: 0.842

4/4 [==============================] - 0s 62ms/step - loss: 1.3174e-04 - categorical_accuracy: 1.0000
Epoch 577/1000
4/4 [==============================] - 0s 64ms/step - loss: 1.3017e-04 - categorical_accuracy: 1.0000
Epoch 578/1000
4/4 [==============================] - 0s 67ms/step - loss: 1.2804e-04 - categorical_accuracy: 1.0000
Epoch 579/1000
4/4 [==============================] - 0s 66ms/step - loss: 1.2631e-04 - categorical_accuracy: 1.0000
Epoch 580/1000
4/4 [==============================] - 0s 67ms/step - loss: 1.2463e-04 - categorical_accuracy: 1.0000
Epoch 581/1000
4/4 [==============================] - 0s 66ms/step - loss: 1.2277e-04 - categorical_accuracy: 1.0000
Epoch 582/1000
4/4 [==============================] - 0s 66ms/step - loss: 1.2087e-04 - categorical_accuracy: 1.0000
Epoch 583/1000
4/4 [==============================] - 0s 63ms/step - loss: 1.1952e-04 - categorical_accuracy: 1.0000
Epoch 584/1000
4/4 [==============================] - 0s 67ms/step - loss: 1.17

4/4 [==============================] - 0s 67ms/step - loss: 3.2819e-05 - categorical_accuracy: 1.0000
Epoch 717/1000
4/4 [==============================] - 0s 64ms/step - loss: 3.2660e-05 - categorical_accuracy: 1.0000
Epoch 718/1000
4/4 [==============================] - 0s 60ms/step - loss: 3.2319e-05 - categorical_accuracy: 1.0000
Epoch 719/1000
4/4 [==============================] - 0s 66ms/step - loss: 3.2079e-05 - categorical_accuracy: 1.0000
Epoch 720/1000
4/4 [==============================] - 0s 67ms/step - loss: 3.1880e-05 - categorical_accuracy: 1.0000
Epoch 721/1000
4/4 [==============================] - 0s 67ms/step - loss: 3.1698e-05 - categorical_accuracy: 1.0000
Epoch 722/1000
4/4 [==============================] - 0s 62ms/step - loss: 3.1445e-05 - categorical_accuracy: 1.0000
Epoch 723/1000
4/4 [==============================] - 0s 70ms/step - loss: 3.1282e-05 - categorical_accuracy: 1.0000
Epoch 724/1000
4/4 [==============================] - 0s 63ms/step - loss: 3.10

4/4 [==============================] - 0s 65ms/step - loss: 1.5310e-05 - categorical_accuracy: 1.0000
Epoch 857/1000
4/4 [==============================] - 0s 68ms/step - loss: 1.5240e-05 - categorical_accuracy: 1.0000
Epoch 858/1000
4/4 [==============================] - 0s 68ms/step - loss: 1.5154e-05 - categorical_accuracy: 1.0000
Epoch 859/1000
4/4 [==============================] - 0s 68ms/step - loss: 1.5086e-05 - categorical_accuracy: 1.0000
Epoch 860/1000
4/4 [==============================] - 0s 68ms/step - loss: 1.5010e-05 - categorical_accuracy: 1.0000
Epoch 861/1000
4/4 [==============================] - 0s 67ms/step - loss: 1.4943e-05 - categorical_accuracy: 1.0000
Epoch 862/1000
4/4 [==============================] - 0s 64ms/step - loss: 1.4895e-05 - categorical_accuracy: 1.0000
Epoch 863/1000
4/4 [==============================] - 0s 71ms/step - loss: 1.4815e-05 - categorical_accuracy: 1.0000
Epoch 864/1000
4/4 [==============================] - 0s 67ms/step - loss: 1.47

4/4 [==============================] - 0s 74ms/step - loss: 8.4540e-06 - categorical_accuracy: 1.0000
Epoch 997/1000
4/4 [==============================] - 0s 72ms/step - loss: 8.4300e-06 - categorical_accuracy: 1.0000
Epoch 998/1000
4/4 [==============================] - 0s 68ms/step - loss: 8.3934e-06 - categorical_accuracy: 1.0000
Epoch 999/1000
4/4 [==============================] - 0s 74ms/step - loss: 8.3568e-06 - categorical_accuracy: 1.0000
Epoch 1000/1000
4/4 [==============================] - 0s 68ms/step - loss: 8.3265e-06 - categorical_accuracy: 1.0000


In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 4)                 132       
                                                                 
Total params: 596,708
Trainable params: 596,708
Non-trai

# 8. Make Predictions

In [29]:
res = model.predict(X_test)

1/1 [==============================] - 1s 591ms/step


In [30]:
actions[np.argmax(res[2])]

'X-Pose'

In [31]:
actions[np.argmax(y_test[2])]

'X-Pose'

# 9. Save Weights

In [32]:
model.save('action.h5')

In [33]:
del model

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [21]:
# Once training is done, u do not have to train it again (no need to call the fit() finction). Just need to setup the model
# and compile it. then load the saved weights.
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [22]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [23]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [24]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [25]:
yhat = model.predict(X_test)

1/1 [==============================] - 1s 698ms/step


In [26]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [27]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 0],
        [0, 2]],

       [[3, 0],
        [0, 3]],

       [[5, 0],
        [0, 1]]], dtype=int64)

In [28]:
accuracy_score(ytrue, yhat)

1.0

# 11. Test in Real Time

In [29]:
from scipy import stats

In [30]:
colors = [(245,117,16), (117,245,16), (16,117,245), (100,100,100)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [35]:
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(res[1])
            
            
        #3. Viz logic
        #    if res[np.argmax(res)] > threshold: 
        #        if len(sentence) > 0: 
        #            if actions[np.argmax(res)] != sentence[-1]:
        #                sentence.append(actions[np.argmax(res)])
        #        else:
        #            sentence.append(actions[np.argmax(res)])

        #    if len(sentence) > 5: 
        #        sentence = sentence[-5:]

         #   # Viz probabilities
         #   image = prob_viz(res, actions, image, colors)
            
        #cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        #cv2.putText(image, ' '.join(sentence), (3,30), 
        #               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        
        if res[1] > threshold:
            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' L Pose Acheived! ', (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.waitKey(10000)
            break

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()